In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("OpenSound/CapTTS-SFT")
ds

/home/nhandt23/anaconda3/envs/speechcraft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['source', 'audio_path', 'caption', 'text', 'speech_duration', 'accent', 'emotion', 'speaking_rate', 'intrinsic_tags', 'emotion_text_prompt', 'all_tags', 'age', 'pitch', 'situational_tags', 'gender', 'basic_tags', 'noise', 'speech_monotony'],
        num_rows: 347783
    })
    val: Dataset({
        features: ['source', 'audio_path', 'caption', 'text', 'speech_duration', 'accent', 'emotion', 'speaking_rate', 'intrinsic_tags', 'emotion_text_prompt', 'all_tags', 'age', 'pitch', 'situational_tags', 'gender', 'basic_tags', 'noise', 'speech_monotony'],
        num_rows: 18348
    })
    test: Dataset({
        features: ['source', 'audio_path', 'caption', 'text', 'speech_duration', 'accent', 'emotion', 'speaking_rate', 'intrinsic_tags', 'emotion_text_prompt', 'all_tags', 'age', 'pitch', 'situational_tags', 'gender', 'basic_tags', 'noise', 'speech_monotony'],
        num_rows: 20756
    })
})

In [2]:
df = ds["train"].to_pandas()
df

,source,audio_path,caption,text,speech_duration,accent,emotion,speaking_rate,intrinsic_tags,emotion_text_prompt,all_tags,age,pitch,situational_tags,gender,basic_tags,noise,speech_monotony
0,vctk,p345_260.wav,An American adult male voice with a dark tone.,What is so unusual about this case?,3.913917,american,None,None,None,None,None,None,None,None,None,None,None,None
1,vctk,p271_197.wav,"A mature man's Scottish voice, masculine and g...",Treasury spokesman Malcolm Bruce is also consi...,5.364458,scottish,None,None,None,None,None,None,None,None,None,None,None,None
2,vctk,p333_173.wav,"A mature American female voice, bright and sli...","Not everyone, however, was pleased with the pr...",3.800917,american,None,None,None,None,None,None,None,None,None,None,None,None
3,vctk,p252_017.wav,A mature man's Scottish voice.,Others have tried to explain the phenomenon ph...,3.862500,scottish,None,None,None,None,None,None,None,None,None,None,None,None
4,vctk,p335_231.wav,"An adult female voice, sounding weak, with a d...","They scored a goal, but we have to keep playing.",4.154125,newzealand,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347778,libritts-r,train-other-500/985/126228/985_126228_000056_0...,"With a slightly relaxed cadence, his words flo...",phillotson seemed not to notice to be surround...,6.160000,None,None,moderate speed,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347779,libritts-r,train-other-500/985/126228/985_126228_000056_0...,The middle-aged man's voice is a unique blend ...,as soon as they had signed their names and c...,5.440000,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347780,libritts-r,train-other-500/985/126228/985_126228_000057_0...,"His speech, while not hurried, is far from let...",the meal at his lodging was a very simple affa...,4.160000,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347781,libritts-r,train-other-500/985/126228/985_126228_000057_0...,"With a subtle hint of dark humor, his voice is...",in crossing the pavement to the fly she looked...,4.680000,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone


In [9]:
df["source"].unique()

array(['vctk', 'emovoice-db', 'voxceleb', 'expresso', 'ears',
       'libritts-r'], dtype=object)

In [12]:
df[df["source"] == "libritts-r"][["audio_path", "text", "caption"]].to_csv("captts_sft.csv", sep="|")

In [16]:
df.iloc[160626]["audio_path"]

'ex03-ex01_laughing_005_channel2_segment_221.12_223.64.wav'

In [24]:
import pandas as pd
import os

# Đường dẫn gốc cho phần READ
base_dir_read = '/data1/speech/nhandt23/06_binh/dataset/expresso/audio_48khz/read/'

# 1. LỌC BỎ CÁC DÒNG CONVERSATIONAL
# Logic: Nếu là source 'expresso' VÀ phần đầu tên file (trước dấu _) có chứa dấu gạch ngang '-'
# Ví dụ: 'ex01-ex02' (có -) -> Conversational -> XÓA
# Ví dụ: 'ex01' (không -) -> Read -> GIỮ

# Tạo mask xác định các dòng là conversational
is_conversational = (df['source'] == 'expresso') & \
                    (df['audio_path'].str.split('_').str[0].str.contains('-'))

# Giữ lại các dòng KHÔNG PHẢI là conversational (dùng dấu ~ để đảo ngược)
df_clean = df[~is_conversational].copy()

# 2. APPLY ĐƯỜNG DẪN TUYỆT ĐỐI CHO FILE READ
def get_read_path(filename):
    try:
        parts = filename.split('_')
        speaker = parts[0] # ex01
        emotion = parts[1] # happy
        # Ghép: base_dir / speaker / emotion / base / filename
        return os.path.join(base_dir_read, speaker, emotion, 'base', filename)
    except:
        return filename

# Apply vào cột mới hoặc ghi đè cột cũ
df_clean['audio_path'] = df_clean.apply(
    lambda row: get_read_path(row['audio_path']) if row['source'] == 'expresso' else row['audio_path'], 
    axis=1
)

# Kiểm tra kết quả
print(f"Số lượng dòng ban đầu: {len(df)}")
print(f"Số lượng dòng sau khi lọc: {len(df_clean)}")
print("-" * 30)
print(df_clean['audio_path'].head())

Số lượng dòng ban đầu: 347783
Số lượng dòng sau khi lọc: 340670
------------------------------
0    p345_260.wav
1    p271_197.wav
2    p333_173.wav
3    p252_017.wav
4    p335_231.wav
Name: audio_path, dtype: object


In [ ]:
new_df = df_clean[df_clean["source"] == "expresso"]
print(len(new_df))
new_df[["audio_path", "text", "caption"]].head(300).to_csv("captts_sft_expresso.txt", sep="|", index=None, header=None)

In [33]:
new_df[["audio_path", "text", "caption"]]

,audio_path,text,caption
44730,/data1/speech/nhandt23/06_binh/dataset/express...,You're covering the medical expenses?,A male speaker's voice is animated and high-p...
44733,/data1/speech/nhandt23/06_binh/dataset/express...,"Oh, yeah, it's like an *insane* drug.","A male speaker's whispery, high-pitched voice..."
44749,/data1/speech/nhandt23/06_binh/dataset/express...,"*Rox*, you're cutting off my circulation!",A female speaker with an American accent sing...
44753,/data1/speech/nhandt23/06_binh/dataset/express...,Um but I'm very curious about what mindfulness...,A male speaker with an American accent delive...
44770,/data1/speech/nhandt23/06_binh/dataset/express...,"Bits of Paradise and ""Save Me The Waltz are re...",A female speaker delivers her words slowly wi...
...,...,...,...
160557,/data1/speech/nhandt23/06_binh/dataset/express...,You trash hotel rooms in your sleep?,A female American speaker delivers her words ...
160589,/data1/speech/nhandt23/06_binh/dataset/express...,You know it's cocaine laced.,A male speaker's voice is characterized by a ...
160615,/data1/speech/nhandt23/06_binh/dataset/express...,"Well, it takes two to tango.",A male speaker with an American accent delive...
160632,/data1/speech/nhandt23/06_binh/dataset/express...,Bring popcorn and a blanket!,"A male speaker delivers an animated, happy pe..."


In [19]:
df.iloc[86332]["audio_path"]

'ex04-ex02_disgusted_013_channel1_segment_59.42_63.65.wav'

In [22]:
new_df.loc[86332]["audio_path_abs"]

'/data1/speech/nhandt23/06_binh/dataset/expresso/audio_48khz/conversational/ex04-ex02/disgusted/ex04-ex02_disgusted_013_channel1_segment_59.42_63.65.wav'

In [ ]:
import os
libritts_r_path = "/data1/speech/cuongph/01_database/librittsr/LibriTTS_R"
new_df = df[df["source"] == "libritts-r"]


new_df["audio_path"] = new_df["audio_path"].apply(lambda x: os.path.join(libritts_r_path, x))

/tmp/ipykernel_986541/2044121638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["source"] == "libritts-r"]["audio_path"] = df[df["source"] == "libritts-r"]["audio_path"].apply(lambda x: os.path.join(libritts_r_path, x))


In [16]:
df[df["source"] == "libritts-r"]

,source,audio_path,caption,text,speech_duration,accent,emotion,speaking_rate,intrinsic_tags,emotion_text_prompt,all_tags,age,pitch,situational_tags,gender,basic_tags,noise,speech_monotony
160675,libritts-r,train-clean-100/103/1241/103_1241_000004_00000...,"With a very feminine, lively, and slightly you...",in fact he had looked at twenty very much as...,5.48,None,None,slightly fast,None,None,None,young adult,moderate pitch,None,female,None,None,slightly expressive and animated
160676,libritts-r,train-clean-100/103/1241/103_1241_000014_00000...,"With a sharp, intellectually engaging tone, sh...",her face was small white and thin also much fr...,9.60,None,None,slightly fast,None,None,None,young adult,moderate pitch,None,female,None,None,monotone
160677,libritts-r,train-clean-100/103/1241/103_1241_000017_00000...,"Her voice, a fusion of youthful exuberance and...",i suppose you are mr matthew cuthbert of gree...,5.32,None,None,slightly fast,None,None,None,young adult,moderate pitch,None,female,None,None,monotone
160678,libritts-r,train-clean-100/103/1241/103_1241_000020_00000...,"Her voice, while slightly adult-like, is also ...",it isn't heavy i've got all my worldly goods...,3.76,None,None,slightly fast,None,None,None,young adult,moderate pitch,None,female,None,None,slightly expressive and animated
160679,libritts-r,train-clean-100/103/1241/103_1241_000021_00000...,"Her voice, though slightly young, was tensed w...",with this matthew's companion stopped talking ...,6.12,None,None,slightly fast,None,None,None,young adult,moderate pitch,None,female,None,None,monotone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347778,libritts-r,train-other-500/985/126228/985_126228_000056_0...,"With a slightly relaxed cadence, his words flo...",phillotson seemed not to notice to be surround...,6.16,None,None,moderate speed,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347779,libritts-r,train-other-500/985/126228/985_126228_000056_0...,The middle-aged man's voice is a unique blend ...,as soon as they had signed their names and c...,5.44,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347780,libritts-r,train-other-500/985/126228/985_126228_000057_0...,"His speech, while not hurried, is far from let...",the meal at his lodging was a very simple affa...,4.16,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
347781,libritts-r,train-other-500/985/126228/985_126228_000057_0...,"With a subtle hint of dark humor, his voice is...",in crossing the pavement to the fly she looked...,4.68,None,None,slightly fast,None,None,None,middle-aged adult,very high-pitch,None,male,None,None,monotone
